In [ ]:
# installing bayesian optimization 
!pip install GPy
!pip install GPyOpt

In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
import tensorflow as tf
import keras
import keras.backend as K
from keras import layers
from keras.models import Sequential,Model
import keras.layers as kl
from keras.layers.merge import concatenate
import gc
import math
import time
import GPy, GPyOpt
import statistics

In [ ]:
# computing RMSE and SMAPE for single and multiple points
def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true), axis=-1))

def single_point_smape(actual, forecast):
    return (100/len(actual)) * (np.sum(2 * np.abs(forecast - actual) / (np.abs(actual) + np.abs(forecast))))
def smape(actual, forecast):
    sum_smape=0
    for i in range(len(actual)):
        sum_smape+=single_point_smape(actual[i],forecast[i])
    return sum_smape/len(actual)

In [ ]:
# creating samples from raw time series
# look_forward is number of points we want to predict from each sample.
# look_back is simply lag size ( window size ). 
def create_dataset(dataset, look_back,look_forward=7):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-look_forward+1):
        a = dataset[i:(i+look_back), 0]
        dataX.append(a)
        a=dataset[(i + look_back):(i + look_back+look_forward), 0]
        dataY.append(a)
    return np.array(dataX), np.array(dataY)

In [ ]:
# creating samples based on features 
def create_dataset_features(dataset, look_back,look_forward=7):
    sample = []
    sample = [float(item) for item in dataset[look_back-5] if math.isnan(item)==False]
    sample=np.array(sample)
    sample = sample.reshape(sample.shape[0],1)
    # number of features are 11, thus we reshape based on this
    sample=sample.reshape(11,int(len(sample)/11))
    sample=sample.T
    sample=np.array(sample)
    sample=sample[0:-look_forward]
    return sample

In [ ]:
# here we need to split train, test, validation
def train_test_split(dataX,dataY,one_sample,look_back,look_forward):
    # test size should be selected manually, here we just select the points not number of samples.
    testsize=35
    train_size=(len(one_sample)-look_forward)-testsize-look_back+1
    
    trainX, testX = dataX[0:train_size,:], dataX[train_size+look_forward-1:,:]
    trainY, testY = dataY[0:train_size,:], dataY[train_size+look_forward-1:,:]
    # validation size should be selected manually, here we just select the points not number of samples.
    valsize=33

    valX, valY=trainX[train_size-valsize+look_forward-1:train_size,:],trainY[train_size-valsize+look_forward-1:train_size]

    trainX = np.reshape(trainX, (trainX.shape[0],1, trainX.shape[1]))
    valX= np.reshape(valX, (valX.shape[0],1, valX.shape[1]))
    testX = np.reshape(testX, (testX.shape[0],1, testX.shape[1]))
    return np.array(trainX),np.array(valX),np.array(testX),np.array(trainY),np.array(valY),np.array(testY)

In [ ]:
# reading the corresponding country csv file. this is confirmed cases dataset.
all_train = pd.read_csv('/kaggle/input/us-covid19-with-feature/US.csv')
all_train=np.array(all_train)
all_train = all_train.flatten()
all_train

In [ ]:
# reading the feature file.
features = pd.read_csv('/kaggle/input/us-covid19-with-feature/US_features.csv')
features=features.T
features=np.array(features)
features

In [ ]:
look_back=0
def final_sample_creation(one_sample,features,lag,look_forward,all_features,):
    # we should define the look_forward variable. this is the number of points we want to predict.
    # our problem is multi-variable prediction and in this dataset the output length is 7. 
    look_forward=7
    look_back=lag

    one_sample=np.array(one_sample)
    one_sample = one_sample.astype('int64')
    one_sample = one_sample.reshape(one_sample.shape[0],1)
    

    dataX_s, dataY_s = create_dataset(one_sample,look_back,look_forward)
    trainX,valX,testX,trainY,valY,testY = train_test_split(dataX_s,dataY_s,one_sample,look_back,look_forward)
    
    dataX_features = create_dataset_features(features,look_back,look_forward)
    dataX_features = dataX_features[:, all_features]
    
    trainX_f = dataX_features[0:len(trainX)]
    valX_f   = dataX_features[len(trainX)-len(valX):len(trainX)]
    testX_f  = dataX_features[-len(testX):]
    
    trainX_f = np.reshape(trainX_f, (trainX_f.shape[0],1, trainX_f.shape[1]))
    valX_f= np.reshape(valX_f, (valX_f.shape[0],1, valX_f.shape[1]))
    testX_f = np.reshape(testX_f, (testX_f.shape[0],1, testX_f.shape[1]))

    return trainX,valX,testX,trainY,valY,testY,trainX_f,valX_f,testX_f

In [ ]:
def HyperParameter_mapping(hyper_parametrs):
    
    learning_rate = hyper_parametrs[0]
    
    
    if hyper_parametrs[1]==0:
        denselayer_activation=None
    elif hyper_parametrs[1]==1:
        denselayer_activation="relu"
    elif hyper_parametrs[1]==2:
        denselayer_activation="linear"
    
    
    
    if hyper_parametrs[2]==0:
        output_activation=None
    elif hyper_parametrs[2]==1:
        output_activation="relu"
    elif hyper_parametrs[2]==2:
        output_activation="linear"
    
    return learning_rate,denselayer_activation,output_activation

In [ ]:
# defining hyper parameters and available range to pass it to the bayesian optimization algorithm
bounds = [{'name': 'Lag',                                     'type': 'discrete',      'domain': (5,6,7,8,9,10,11,12,13,14,15)},
          {'name': 'Learning_rate',                           'type': 'discrete',    'domain': (0.0001,0.0005,0.001,0.005,0.01,0.05)},
          {'name': 'dense_activation',                        'type': 'discrete',      'domain': (0,1,2)},
          {'name': 'output_activation',                       'type': 'discrete',      'domain': (0,1,2)},
         ]

In [ ]:
# saving parameters of the trained models
def write_to_file_ATT(file_name,lag,learning_rate,denselayer_activation,output_activation,final_smape,final_rmse,final_val_RMSE,epoch):
    fname=""+file_name +".txt"
    f1=open(fname,"a+")
    f1.write("lag=")
    f1.write(str(lag))
    f1.write(",learning_rate=")
    f1.write(str(learning_rate))
    f1.write(",denselayer_activation=")
    f1.write(str(denselayer_activation))
    f1.write(",output_activation=")
    f1.write(str(output_activation))
    f1.write(",final_smape=")
    f1.write(str(final_smape))
    f1.write(",final_rmse=")
    f1.write(str(final_rmse))
    f1.write(",final_val_RMSE=")
    f1.write(str(final_val_RMSE))
    f1.write(",epoch=")
    f1.write(str(epoch))
    f1.write("\n")
    f1.close()

In [ ]:
global_min_val_RMSE = math.inf
# the function that we want to minimize based on the beysian optimization.
# here we define our NN model. input parameters will be given to this function.
def Main(x):
    # first parameter is lag size 
    lag=int(x[:,0])
    
    look_forward=7
    # feature indexes for each lag size, for example [1, 3, 6] means for lag size 6 the
    # features indexes we will use are 1,3 and 6. these indexes should be find with the
    # feature selection JaGen algorithm and then manually must be written here.
    all_features = [[4],
                    [1, 3, 6],
                    [3],
                    [0, 1, 8, 9, 10],
                    [6, 7],
                    [6],
                    [0, 3, 5, 10],
                    [0, 1, 3, 10],
                    [1],
                    [1, 5],
                    [1]
                   ]

    hyper_parametrs=x[:,1:][0]
    all_features = all_features [lag-5]
    
    # here we need to recall HyperParameter_mapping to get model hyperparameters 
    # and recall final_sample_creation to create samples
    learning_rate,denselayer_activation,output_activation=HyperParameter_mapping(hyper_parametrs)
    trainX,valX,testX,trainY,valY,testY,trainX_f,valX_f,testX_f=final_sample_creation(all_train,features,lag,look_forward,all_features)
    
    # initialize all metrices to zero
    RMSE_loss_stdev =[]
    SMAPE_loss_stdev =[]
    test_RMSE=0
    test_SMAPE=0
    RMSE_SUM = 0
    SMAPE_SUM = 0
    val_RMSE=0
    val_RMSE_sum=0
    predicted_samples = np.zeros(math.ceil(len(testX)/7) *7)
    
    global global_min_val_RMSE
    input2_len=int(len(trainX_f[0][0]))
    
    # performing 10 iteration at each step
    for j in range(10):
        # defining  NN layers
        input1 = layers.Input(shape=(1,lag,))
        input2 = layers.Input(shape=(1,input2_len,))
        
        multi_head1 = kl.MultiHeadAttention(key_dim=1,num_heads=lag,name='Multi-Head1')(input1,input1,input1)
        
        multi_head2 = kl.MultiHeadAttention(key_dim=1,num_heads=input2_len,name='Multi-Head2')(input2,input2,input2)                    
    
        flatt1 = kl.Flatten()(multi_head1)
        flatt2 = kl.Flatten()(multi_head2) 
        concat=concatenate([flatt1,flatt2])
                          
        fully_connected1 = kl.Dense(100,activation=denselayer_activation)(concat)

        fully_connected2 = kl.Dense(look_forward,activation=output_activation)(fully_connected1)
        # create model
        model = Model(inputs=[input1,input2],outputs=fully_connected2)
        # define the optimization
        opt=tf.keras.optimizers.Adam(lr=learning_rate)
        # complile the model
        model.compile(loss="mse",optimizer=opt,metrics=["mse"])
        # define count variable to perform early stopping method
        count=0
        # min_RMSE variable is for saving minimum validation RMSE because of controlling early stopping
        min_RMSE=0
        # here manually we perform a loop to run the model 500 times (epoch size =500)
        for k in range(500):
            history=model.fit([trainX,trainX_f],trainY,
                verbose=0,
                batch_size=1,
                )
            count+=1
           
            results = model.predict([valX,valX_f],batch_size=16, verbose=0)
            results = results.astype('int64')
            results = results.round()
            val_RMSE = math.sqrt(mean_squared_error(valY, results[:]))
            
            if k==0:
                min_RMSE=val_RMSE
                results = model.predict([testX,testX_f],batch_size=16, verbose=0)
                results = results.astype('int64')
                results = results.round()
                # we want predict 7 points at each step. but our window moving step is 1.
                # thus we can predict all and then pick every 7 samples to calculate Metrics on test.
                r_y=results[0: : 7]
                t_y=testY[0: : 7]
                test_RMSE = math.sqrt(mean_squared_error(t_y, r_y[:]))
                test_SMAPE = smape(t_y, r_y[:])
            if k>0 and val_RMSE<=min_RMSE:
                # if new val_RMSE finde, then put count to zero
                count=0
                min_RMSE=val_RMSE
                results = model.predict([testX,testX_f],batch_size=16, verbose=0)
                results = results.astype('int64')
                results = results.round()
                r_y=results[0: : 7]
                t_y=testY[0: : 7]
                test_RMSE = math.sqrt(mean_squared_error(t_y, r_y[:]))
                test_SMAPE = smape(t_y, r_y[:])
            # if after 50 epochs val_RMSE whouldn't improved, then break the loop.    
            if count>50:
                break
        # saving predicted samples 
        predicted_samples += r_y.ravel()        
        # saving RMSE and SMAPE of each iteration for calculating standard deviation and final loss
        RMSE_SUM += test_RMSE
        RMSE_loss_stdev.append(test_RMSE)
        SMAPE_SUM += test_SMAPE
        SMAPE_loss_stdev.append(test_SMAPE)
        val_RMSE_sum +=min_RMSE
        K.clear_session()
        gc.collect()
        model=None
        del history
    if math.isnan(min_RMSE) or SMAPE_SUM==0:
        final_smape=math.inf
        final_rmse=math.inf
        final_val_RMSE=math.inf
    else:
        final_smape = SMAPE_SUM/10
        final_rmse = RMSE_SUM/10
        final_val_RMSE=val_RMSE_sum/10
    
    print("\n\n\n Finished \n\n\n")
    print("SMAPE:\t{0} \t RMSE:\t{1}".format(final_smape, final_rmse))
    print("val_RMSE:\t{0}".format(final_val_RMSE))
    # saving all parameters and metrics in file.
    file_name="US_att_feature_params"
    write_to_file_ATT(file_name,lag,learning_rate,denselayer_activation,output_activation,final_smape,final_rmse,final_val_RMSE,k)
    if final_val_RMSE<=global_min_val_RMSE:
        global_min_val_RMSE = final_val_RMSE
        # writing the prediction points in a txt file.
        f1=open("outputs_att_US_feature.txt","w")
        x = predicted_samples.ravel()
        x=x/10
        for i in x:
            f1.write(str(i))
            f1.write("\n") 
    # calculating STD and printing all loss values.
    print(RMSE_loss_stdev)
    print(statistics.stdev(RMSE_loss_stdev))
    print(SMAPE_loss_stdev)
    print(statistics.stdev(SMAPE_loss_stdev))
    return final_val_RMSE 

In [ ]:
opt_us = GPyOpt.methods.BayesianOptimization(f=Main, domain=bounds)

In [ ]:
# running bayesian optimization for 50 times.
opt_us.run_optimization(max_iter=50)

In [ ]:
print("optimized loss: {0}".format(opt_us.fx_opt))

In [ ]:
# best parameters
opt_us.x_opt

In [ ]:
print("""
Optimized Parameters:
\t{0}:\t{1}
\t{2}:\t{3}
\t{4}:\t{5}
\t{6}:\t{7}
""".format(bounds[0]["name"],opt_us.x_opt[0],
           bounds[1]["name"],opt_us.x_opt[1],
           bounds[2]["name"],opt_us.x_opt[2],
           bounds[3]["name"],opt_us.x_opt[3],
           ))
print("optimized loss: {0}".format(opt_us.fx_opt))